In [ ]:
#!pip -q install git+https://github.com/huggingface/transformers

In [ ]:
#!pip install tiktoken chromadb langchain

In [7]:
# from git import Repo
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language

In [8]:
# Load
loader = GenericLoader.from_filesystem(
    "/home/ec2-user/SageMaker/VegRD_APD_Fruit_Phenotyping/scripts",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=5000),
)
documents = loader.load()
len(documents)

35

In [9]:
len(documents)

35

In [10]:
documents[0]

Document(page_content='import numpy as np\nimport os, sys\nfrom collections import defaultdict\nfrom get_data import get_images\nimport re\n\ndef update_config(config_defaults,**kwargs):\n    new_config = config_defaults.copy()\n    for key, value in kwargs.items():\n        if key in new_config:\n            new_config[key] = value\n        else:\n            raise KeyError(f"Unknown configuration key: {key}")\n    return new_config\n\n\ndef pattern_exists_in_string(string, pattern="BLK"):\n    if re.search(pattern, string):\n        return True\n    else:\n        return False\n\n\n# A dictionary of projects and images\nprojs = defaultdict(list)\n\nwith open("new_images.txt", "r") as f:\n    for l in f.readlines():\n        l=l.strip()\n        splitted= l.split("/")\n        projs[splitted[1]].append(l)\n\nfor k,v in projs.items():\n    get_images(k,s3_bucket="model-deployment-automation", files_to_copy=v)\n    if pattern_exists_in_string(k,pattern="BLK"):\n        from config impor

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, chunk_size=2000, chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

199

In [12]:
#!pip install langchain sentence_transformers

In [13]:
#from langchain.embeddings import HuggingFaceInstructEmbeddings
#embeddings = HuggingFaceInstructEmbeddings(
#    query_instruction="Query about the code base: ",
#    model_kwargs = {'device': 'cpu'}
#)

In [14]:
#!pip install langchain sentence_transformers

from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
#from langchain.embeddings import HuggingFaceBgeEmbeddings

#model_name = "BAAI/BAAI/bge-base-en"
#model_kwargs = {"device": "cpu"}
#encode_kwargs = {"normalize_embeddings": True}
#hf = HuggingFaceBgeEmbeddings(
#    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
#)

In [16]:
#document_embeddings = [embeddings.embed_documents(text.page_content) for text in texts]
from langchain.vectorstores import Chroma
# Create a vector store
#vector_store = Chroma.from_embeddings(document_embeddings)
db = Chroma.from_documents(documents, embeddings)

In [17]:
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.vectorstores import Chroma

#db = Chroma.from_documents(texts, embeddings)
#retriever = db.as_retriever(
#    search_type="mmr",  # Also test "similarity"
#    search_kwargs={"k": 8},
#)

In [18]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory

In [19]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

device="cpu"

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype= torch.float32,device_map=device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000,temperature=0.1, do_sample=True)
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# To check the device and dtype
# Get the first parameter of the model and check its device and dtype
first_param = next(model.parameters())
model_device = first_param.device
model_dtype = first_param.dtype

print(f"Model is on device: {model_device}")
print(f"Model data type: {model_dtype}")

In [ ]:
## Test LLM without Context

#from langchain.prompts import PromptTemplate

#template = """Question: {question}

#Answer: Let's think step by step."""
#prompt = PromptTemplate.from_template(template)

#chain = prompt | llm

#question = "What is electroencephalography?"

#print(chain.invoke({"question": question}))

In [ ]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=False)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(),memory=memory)

In [1]:
#!pip install langchainhub

  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.34.0 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.1.0 which is incompatible.
sagemaker 2.200.0 requires urllib3<1.27, but you have urllib3 2.1.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [20]:
question = "How is curved backbone of hot peppers calculated?"
docs = db.similarity_search(question)

In [ ]:
from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")
from langchain.chains.question_answering import load_qa_chain

# Chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=rag_prompt)
chain({"input_documents": docs, "question": question}, return_only_outputs=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [4]:
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
question = "How is curved backbone of hot peppers calculated??"
result = qa(question)
result["answer"]

In [ ]:
questions = [
    "How is curved backbone of hot peppers calculated?",
    "How to turn on/off visulaizations of phenotypes?",
    "How to turn on/off visulaizations of detections and phenotypes of hot peppers lsit all methods?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")